In [32]:
!python -V

Python 3.9.19


In [33]:
import pandas as pd

In [34]:
import pickle

In [35]:
import seaborn as sns
import matplotlib.pyplot as plt

In [36]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [37]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/son/Documents/GitHub/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1716546824066, experiment_id='1', last_update_time=1716546824066, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [38]:
def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [39]:
df_train = read_dataframe('./data/green_tripdata_2021-01.csv')
df_val = read_dataframe('./data/green_tripdata_2021-02.csv')

/var/folders/dl/74t5rlpn7yg9n8p83fsrhfrh0000gn/T/ipykernel_12182/3970424726.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [40]:
len(df_train), len(df_val)

(73908, 61921)

In [41]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [42]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [43]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [44]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

2024/05/26 10:58:15 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '19ce86726b7c454b9fafc9b964dd5632', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/05/26 10:58:15 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


7.7587152073244585

In [46]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [47]:
with mlflow.start_run():

    mlflow.set_tag("developer", "sonle")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

2024/05/26 10:59:47 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


In [18]:
import xgboost as xgb

In [19]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [20]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [21]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [22]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:10.86050                          
[1]	validation-rmse:9.81412                           
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [07:21:53] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[2]	validation-rmse:9.01685                           
[3]	validation-rmse:8.41510                           
[4]	validation-rmse:7.96585                           
[5]	validation-rmse:7.63118                           
[6]	validation-rmse:7.38517                           
[7]	validation-rmse:7.20287                           
[8]	validation-rmse:7.06826                           
[9]	validation-rmse:6.96944                           
[10]	validation-rmse:6.89447                          
[11]	validation-rmse:6.83738                          
[12]	validation-rmse:6.79351                          
[13]	validation-rmse:6.75985                          
[14]	validation-rmse:6.73290                          
[15]	validation-rmse:6.71466                          
[16]	validation-rmse:6.69851                          
[17]	validation-rmse:6.68498                          
[18]	validation-rmse:6.67468                          
[19]	validation-rmse:6.66669                          
[20]	valid

/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [07:22:17] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[4]	validation-rmse:8.89713                                                    
[5]	validation-rmse:8.51667                                                    
[6]	validation-rmse:8.20136                                                    
[7]	validation-rmse:7.94089                                                    
[8]	validation-rmse:7.72609                                                    
[9]	validation-rmse:7.54973                                                    
[10]	validation-rmse:7.40500                                                   
[11]	validation-rmse:7.28636                                                   
[12]	validation-rmse:7.18880                                                   
[13]	validation-rmse:7.10875                                                   
[14]	validation-rmse:7.04340                                                   
[15]	validation-rmse:6.98912                                                   
[16]	validation-rmse:6.94338            

...

[1]	validation-rmse:7.27347                                                     
[2]	validation-rmse:6.83811                                                     
[3]	validation-rmse:6.68274                                                     
[4]	validation-rmse:6.61032                                                     
[5]	validation-rmse:6.57560                                                     
[6]	validation-rmse:6.55483                                                     
[7]	validation-rmse:6.54048                                                     
[8]	validation-rmse:6.52442                                                     
[9]	validation-rmse:6.51991                                                     
[10]	validation-rmse:6.51634                                                    
[11]	validation-rmse:6.51324                                                    
[12]	validation-rmse:6.50823                                                    
[13]	validation-rmse:6.50495

/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [07:44:54] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.07377                                                     
[1]	validation-rmse:6.60945                                                     
[2]	validation-rmse:6.54259                                                     
[3]	validation-rmse:6.51982                                                     
[4]	validation-rmse:6.51007                                                     
[5]	validation-rmse:6.49757                                                     
[6]	validation-rmse:6.49047                                                     
[7]	validation-rmse:6.48084                                                     
[8]	validation-rmse:6.47296                                                     
[9]	validation-rmse:6.46205                                                     
[10]	validation-rmse:6.45563                                                    
[11]	validation-rmse:6.45205                                                    
[12]	validation-rmse:6.44519

/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [07:45:01] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.37488                                                    
[1]	validation-rmse:10.65072                                                    
[2]	validation-rmse:10.03186                                                    
[3]	validation-rmse:9.49350                                                     
[4]	validation-rmse:9.04472                                                     
[5]	validation-rmse:8.65943                                                     
[6]	validation-rmse:8.33383                                                     
[7]	validation-rmse:8.05216                                                     
[8]	validation-rmse:7.82265                                                     
[9]	validation-rmse:7.62243                                                     
[10]	validation-rmse:7.45504                                                    
[11]	validation-rmse:7.31705                                                    
[12]	validation-rmse:7.20519

In [49]:
mlflow.xgboost.autolog(disable=True)

In [50]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [11:08:11] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23166
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23597
[8]	validation-rmse:7.99320
[9]	validation-rmse:7.78709
[10]	validation-rmse:7.61022
[11]	validation-rmse:7.45952
[12]	validation-rmse:7.33049
[13]	validation-rmse:7.22098
[14]	validation-rmse:7.12713
[15]	validation-rmse:7.04752
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87112
[19]	validation-rmse:6.82740
[20]	validation-rmse:6.78995
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72994
[23]	validation-rmse:6.70547
[24]	validation-rmse:6.68390
[25]	validation-rmse:6.66421
[26]	validation-rmse:6.64806
[27]	validation-rmse:6.63280
[28]	validation-rmse:6.61924
[29]	validation-rmse:6.60773
[30]	validation-rmse:6.59777
[31]	validation-rmse:6.58875
[32]	validation-rmse:6.58107
[33]	validation-rmse:6.57217
[34]	validation-rmse:

/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [11:08:45] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [51]:
logged_model = 'runs:/7fdb8da0da054dad8ccc9888fafd81fb/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [11:12:33] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [52]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 7fdb8da0da054dad8ccc9888fafd81fb

In [53]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [11:13:38] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [54]:
xgboost_model

In [55]:
y_pred = xgboost_model.predict(valid)

In [56]:
y_pred[:10]

array([14.782765 ,  7.184751 , 15.971323 , 24.328938 ,  9.559302 ,
       17.115105 , 11.6522455,  8.688133 ,  8.962229 , 18.982166 ],
      dtype=float32)

In [25]:
# !python -m pip install scikit-learn==1.3.2

In [30]:
# import sklearn
# print('The scikit-learn version is {}.'.format(sklearn.__version__))

# import mlflow
# print('The mlflow version is {}.'.format(mlflow.__version__))

The scikit-learn version is 1.3.2.
The mlflow version is 2.13.0.


In [57]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2024/05/26 11:16:49 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/26 11:24:27 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/26 11:24:34 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/26 11:45:58 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.